In [17]:

import json
from datetime import date, datetime, timedelta
from time import time
import pytz



limit1 = datetime.strptime("9:00:00", "%H:%M:%S").time()
limit2 = datetime.strptime("20:00:00", "%H:%M:%S").time()
limit3 = datetime.strptime("12:00:00", "%H:%M:%S").time()



doctor_time_tables = {
    "Dr. João Santos": {"monday": {'start': '09:00:00', 'end': "13:00:00"}, "wednesday": {'start': '18:00:00','end': "20:00:00"}},
    "Dr. Miguel Costa": {"monday": {'start': "13:30:00", 'end': "17:30:00"}, "thursday": {'start': '09:00:00', 'end': "13:00:00"}},
    "Dra. Sofia Pereira": {"monday": {'start': "18:00:00", 'end': "20:00:00"}, "thursday": {'start': '13:30:00', 'end': "17:30:00"}},
    "Dra. Mariana Chagas": {"tuesday": {'start': "09:00:00", 'end': "13:00:00"}, "thursday": {'start': '18:00:00', 'end': "20:00:00"}},
    "Dr. António Oliveira": {"tuesday": {'start': "13:30:00", 'end': "17:30:00"}, "friday": {'start': '09:00:00', 'end': "13:00:00"}},
    "Dra. Inês Martins": {"tuesday": {'start': "18:00:00", 'end': "20:00:00"}, "friday": {'start': '13:30:00', 'end': "17:30:00"}},
    "Dr. Ângelo Rodrigues": {"wednesday": {'start': "09:00:00", 'end': "13:00:00"}, "friday": {'start': '18:00:00', 'end': "20:00:00"}},
    "Dr. José Dias": {"wednesday": {'start': "13:30:00", 'end': "17:30:00"}, "saturday": {'start': '09:00:00', 'end': "12:00:00"}},
}


In [18]:
def is_time_in_range(selected_time, time_list):
    selected_time = datetime.strptime(selected_time, "%H:%M:%S").time()
    
    for time in time_list:
        time_obj = datetime.strptime(time, "%H:%M:%S").time()
        if time_obj == selected_time:
            return True
    return False



In [19]:
def get_available_time_slots(doctor, day):
    # Function to get available time slots for a specific doctor on a given day
    doctor_schedule = doctor_time_tables.get(doctor, {}).get(day, {})
    
    if not doctor_schedule:
        return []

    start_time = datetime.strptime(doctor_schedule.get('start', '00:00:00'), "%H:%M:%S").time()
    end_time = datetime.strptime(doctor_schedule.get('end', '23:59:59'), "%H:%M:%S").time()

    available_times = []
    current_time = start_time

    while current_time <= end_time:
        available_times.append(current_time.strftime("%H:%M:%S"))
        current_time = (datetime.combine(date.today(), current_time) + timedelta(minutes=30)).time()

    return available_times

In [20]:
def get_available_times_message(doctor, day):
    available_times = get_available_time_slots(doctor, day)
    if not available_times:
        return f"{doctor} is not available on {day}."
    return f"{doctor} is available on {day} at the following times: {', '.join(available_times)}."


In [21]:
def choose_doctor_based_on_time(requested_time_str, requested_doctor, requested_date_str):
    available_doctors = []
    requested_date = datetime.strptime(requested_date_str, "%Y-%m-%d").strftime("%A").lower()   # Monday
    requested_time = datetime.strptime(requested_time_str, "%H:%M:%S").time()  # 09:00:00

    for doctor, schedules in doctor_time_tables.items():
        for day, time_range in schedules.items():
            if day == requested_date:
                start_time = datetime.strptime(time_range['start'], "%H:%M:%S").time()
                end_time = datetime.strptime(time_range['end'], "%H:%M:%S").time()

                if start_time <= requested_time <= end_time:
                    available_doctors.append(doctor)

    if not available_doctors:
        return None  # No available doctors for the requested time

    if requested_doctor in available_doctors:
        return requested_doctor

    return None  # The originally reques

In [22]:
def doctor_checking(arguments):
    provided_date =  str(datetime.strptime(json.loads(arguments)['date'], "%Y-%m-%d").date())
    provided_time = str(datetime.strptime(json.loads(arguments)['time'].replace("PM","").replace("AM","").strip(), "%H:%M:%S").time())
    start_date_time = provided_date + " " + provided_time
    timezone = pytz.timezone('Europe/Lisbon')
    start_date_time = timezone.localize(datetime.strptime(start_date_time, "%Y-%m-%d %H:%M:%S"))
    email_address = json.loads(arguments)['email_address']
    doctor = json.loads(arguments)['doctor']
    end_date_time = start_date_time + timedelta(hours=0.5)

    
    existing_doctors = ["Dr. João Santos", "Dr. Miguel Costa", "Dra. Sofia Pereira", 'Dra. Mariana Chagas', 'Dr. António Oliveira',
                            'Dra. Inês Martins', 'Dr. Ângelo Rodrigues', 'Dr. José Dias']


    if doctor is not None:
        if doctor not in existing_doctors:
            return "Please choose a valid doctor from the list: " + ", ".join(existing_doctors)
                
        day_of_week = start_date_time.strftime("%A").lower()
        doctor_schedule = get_available_time_slots(doctor, day_of_week)

        if not doctor_schedule or not is_time_in_range(provided_time, doctor_schedule):
            available_times_message = get_available_times_message(doctor, day_of_week)
            
            return f"{doctor} is not available at the selected time. {available_times_message} Please choose a valid time."
        else:
            # Choose a doctor based on the requested time
            requested_time = start_date_time.time().strftime("%H:%M:%S")
            updated_doctor = choose_doctor_based_on_time(requested_time, doctor, provided_date)
            if updated_doctor is None:
                return False
            # Update the 'doctor' variable with the chosen doctor
            #doctor = updated_doctor
            # Confirm the appointment
            return True



In [23]:
doctor_schedule = get_available_time_slots('Dr. João Santos', 'monday')

In [24]:
is_time_in_range('17:00:00', doctor_schedule)

False

In [25]:
timezone = pytz.timezone('Europe/Lisbon')
doctor = 'Dr. João Santos'
provided_time = '17:00:00'
provided_date = datetime.strptime('2024-01-01', "%Y-%m-%d").date()
start_date_time = datetime.combine(provided_date, datetime.strptime(provided_time, "%H:%M:%S").time())
start_date_time = timezone.localize(start_date_time)


if not doctor_schedule or not is_time_in_range(provided_time, doctor_schedule):
        available_times_message = get_available_times_message(doctor, 'monday')
        print(f"{doctor} is not available at the selected time. {available_times_message} Please choose a valid time.")
else:
        # Choose a doctor based on the requested time
        requested_time = start_date_time.time().strftime("%H:%M:%S")
        updated_doctor = choose_doctor_based_on_time(requested_time, doctor, provided_date)
        if updated_doctor is None:
                print(False)

Dr. João Santos is not available at the selected time. Dr. João Santos is available on monday at the following times: 09:00:00, 09:30:00, 10:00:00, 10:30:00, 11:00:00, 11:30:00, 12:00:00, 12:30:00, 13:00:00. Please choose a valid time.


In [27]:
updated_doctor = choose_doctor_based_on_time('09:00:00', 'Dr. João Santos', '2024-01-01')
print(updated_doctor)

Dr. João Santos


In [1]:
import json
from datetime import date, datetime, timedelta
from time import time
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
import pytz


In [2]:
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [3]:
SCOPES = ['https://www.googleapis.com/auth/calendar']
creds = Credentials.from_authorized_user_file('token.json', SCOPES)
service = build('calendar', 'v3', credentials=creds)

In [4]:

limit1 = datetime.strptime("9:00:00", "%H:%M:%S").time()
limit2 = datetime.strptime("20:00:00", "%H:%M:%S").time()
limit3 = datetime.strptime("12:00:00", "%H:%M:%S").time()

timezone = pytz.timezone('Europe/Lisbon')
doctor = 'Dr. João Santos'
provided_time = '13:00:00'
provided_date = datetime.strptime('2024-01-01', "%Y-%m-%d").date()
start_date_time = datetime.combine(provided_date, datetime.strptime(provided_time, "%H:%M:%S").time())
start_date_time = timezone.localize(start_date_time)
email_address = 'catioli@hotmail.com'


if start_date_time.time() >= limit1 and start_date_time.time() <= limit2: #Day of the week
    end_date_time = start_date_time + timedelta(hours=2)
    events = service.events().list(calendarId="primary").execute()
    id = ""
    final_event = None
    for event in events['items']:
        print(event)
        if event['attendees'][0]['email'] == email_address:
            id = event['id']
            final_event = event


{'kind': 'calendar#event', 'etag': '"3407962101988000"', 'id': 'it9jljdhocv7bj793rntt87ops', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=aXQ5amxqZGhvY3Y3Ymo3OTNybnR0ODdvcHMgZG9jLml0LnJpZ2h0LmNwQG0', 'created': '2023-12-31T00:04:10.000Z', 'updated': '2023-12-31T00:04:10.994Z', 'summary': 'Appointment for Dr. José Dias', 'description': 'This appointment has been scheduled by the booking chatbot', 'location': 'Lisbon', 'creator': {'email': 'doc.it.right.cp@gmail.com', 'self': True}, 'organizer': {'email': 'doc.it.right.cp@gmail.com', 'self': True}, 'start': {'dateTime': '2024-01-03T13:30:00Z', 'timeZone': 'Europe/Lisbon'}, 'end': {'dateTime': '2024-01-03T14:00:00Z', 'timeZone': 'Europe/Lisbon'}, 'iCalUID': 'it9jljdhocv7bj793rntt87ops@google.com', 'sequence': 0, 'attendees': [{'email': 'ruiizy7@gmail.com', 'responseStatus': 'needsAction'}], 'reminders': {'useDefault': False, 'overrides': [{'method': 'popup', 'minutes': 10}, {'method': 'email', 'minutes': 1

In [53]:
def check_event(email_address):
    
    events = service.events().list(calendarId="primary").execute()  #all the events presented in google calendar
    id = ""
    final_event = None
    events_list = []
    for event in events['items']:
        print(event['attendees'][0]['email'])

        if event['attendees'][0]['email'] == email_address: #if the email exists in the attendees 
            id = event['id'] #id => event id
            final_event = event #final event => is the full event
    if final_event:
        start_datetime_str = final_event['start']['dateTime']
        start_datetime = datetime.fromisoformat(start_datetime_str[:-1]).replace(tzinfo=pytz.UTC)
        formatted_date= start_datetime.strftime('%Y-%m-%d')
        formatted_time = start_datetime.strftime('%H:%M:%S')


        return f"Your appointment is right now scheduled for {formatted_date} at {formatted_time} with {final_event['summary']} do you wish to keep the same doctor?"      
    else:
        return f"Your email is not assigned to any appointment. Please try again."


In [54]:
email_address = 'catioli@hotmail.com'


checking = check_event('catioli@hotmail.com')
checking

ruiizy7@gmail.com
catioli@hotmail.com
catioli@hotmail.com
example@example.com
catioli@hotmail.com
example@example.com


'Your appointment is right now scheduled for 2024-01-01 at 10:30:00 with Dr. João Santos do you wish to keep the same doctor?'

In [ ]:
def appointment_reschedule(arguments):

    try:
        email_address = json.loads(arguments)['email_address']
        provided_date =  str(datetime.strptime(json.loads(arguments)['date'], "%Y-%m-%d").date())
        provided_time = str(datetime.strptime(json.loads(arguments)['time'].replace("PM","").replace("AM","").strip(), "%H:%M:%S").time())
        start_date_time = provided_date + " " + provided_time
        timezone = pytz.timezone('Europe/Lisbon')
        start_date_time = timezone.localize(datetime.strptime(start_date_time, "%Y-%m-%d %H:%M:%S"))
        


        existing_doctors = ["Dr. João Santos", "Dr. Miguel Costa", "Dra. Sofia Pereira", 'Dra. Mariana Chagas', 'Dr. António Oliveira',
                            'Dra. Inês Martins', 'Dr. Ângelo Rodrigues', 'Dr. José Dias']
        
        # check if the event exists and conclude if the user wants to keep the same doctor
        checking = check_event(arguments)
        user_response = chat_completion_request(checking)
        
        # if the user answers yes
        if "yes" in user_response.lower():

            if provided_date and provided_time:

            #Confirm that the new date is available in the calendar and with the doctor
                slot_checking = appointment_checking(arguments) 
                doc_checking = doctor_checking(arguments, predetermined_doctor = final_event['summary'])

                while doc_checking != True:
                    return doc_checking        

                while slot_checking != True:
                    return slot_checking   
            
                # if the new date is available and correct
                if doc_checking == True and slot_checking == True:        

                    event = {
                        'summary': {final_event['summary']},
                        'location': "Lisbon",
                        'description': "{This appointment was scheduled by the chatbot}",
                        
                        'start': {
                            'dateTime': start_date_time.strftime("%Y-%m-%dT%H:%M:%S"),
                            'timeZone': 'Europe/Lisbon',
                        },
                        'end': {
                            'dateTime': end_date_time.strftime("%Y-%m-%dT%H:%M:%S"),
                            'timeZone': 'Europe/Lisbon',
                        },
                        'attendees': [
                            {'email': email_address},
                        ],
                                            
                        'reminders': {
                            'useDefault': False,
                            'overrides': [
                                {'method': 'email', 'minutes': 24 * 60},
                                {'method': 'popup', 'minutes': 10},
                            ],
                        },
                    }
                    service.events().insert(calendarId='primary', body=event).execute()
                    return "Appointment added successfully."
            else:
                 return "Please provide date and time."
                
        if "no" in user_response.lower():

            #choose other doctor
            doctor = json.loads(arguments)['doctor']

            if provided_date and provided_time and doctor:

                slot_checking = appointment_checking(arguments) 
                doc_checking = doctor_checking(arguments)

                while doc_checking != True:
                    return doc_checking        

                while slot_checking != True:
                    return slot_checking   
        
                if doc_checking == True and slot_checking == True:        
                    
                    event = {
                        'summary': {doctor},
                        'location': "Lisbon",
                        'description': "{This appointment was scheduled by the chatbot}",
                        
                        'start': {
                            'dateTime': start_date_time.strftime("%Y-%m-%dT%H:%M:%S"),
                            'timeZone': 'Europe/Lisbon',
                        },
                        'end': {
                            'dateTime': end_date_time.strftime("%Y-%m-%dT%H:%M:%S"),
                            'timeZone': 'Europe/Lisbon',
                        },
                        'attendees': [
                            {'email': email_address},
                        ],
                                            
                        'reminders': {
                            'useDefault': False,
                            'overrides': [
                                {'method': 'email', 'minutes': 24 * 60},
                                {'method': 'popup', 'minutes': 10},
                            ],
                        },
                    }
                    service.events().insert(calendarId='primary', body=event).execute()
                    return "Appointment added successfully."              
            else:
                 return "Please provide date, time and doctor of preference."
        else:
            return "Please answer yes or no"
        
    except Exception as e:
            import traceback
            traceback.print_exc()
            return "We are enable to process. Please try again."

In [ ]:

       
        # # Fetch response of ChatGPT and extract content
        # assistant_message = chat_response.json().get("choices", [{}])[0].get("message", {})

        # if assistant_message.get('content'):
        #     user_response = assistant_message['content']
        #     print("Response is: ", user_response)
        #     messages.append({"role": "assistant", "content": user_response})
        # else:
        #     # Handle the case when the response contains a function call
        #     fn_name = assistant_message.get("function_call", {}).get("name", "")
        #     arguments = assistant_message.get("function_call", {}).get("arguments", "")
        #     function = locals().get(fn_name, None)
            
        #     if function:
        #         result = function(arguments)
        #         print("Response is: ", result)
        #         # You might need to handle the result further based on your use case
        #         user_response = None  # You might need
        #         # if the user answers yes
